In [2]:
#
# Martin Boldt, Blekinge Institute of Technology, Oct 2019.
#

import pandas as pd
import numpy as np

### Läs in datan från filen "Grilla.csv" till en Pandas Dataframe och skriv ut datan på skärmen. Tolka varje kolumn som kategorisk data (engelska: 'category')

In [3]:
df = pd.read_csv("Grilla.csv",  dtype='category', sep=",")
print( df )

      Dag  Väder Temperatur Fuktighet   Vind Grilla
0   Dag01    Sol      Varmt       Hög   Svag     Ja
1   Dag02    Sol      Varmt       Hög  Stark     Ja
2   Dag03  Mulet      Varmt       Hög   Svag     Ja
3   Dag04   Regn      Varmt       Hög   Svag     Ja
4   Dag05   Regn      Kallt    Normal   Svag    Nej
5   Dag06   Regn      Kallt    Normal  Stark    Nej
6   Dag07  Mulet      Kallt    Normal   Svag     Ja
7   Dag08    Sol       Milt       Hög   Svag     Ja
8   Dag09    Sol      Kallt    Normal   Svag     Ja
9   Dag10   Regn       Milt    Normal   Svag    Nej
10  Dag11    Sol       Milt    Normal  Stark     Ja
11  Dag12  Mulet       Milt       Hög  Stark    Nej
12  Dag13  Mulet      Varmt    Normal   Svag     Ja
13  Dag14   Regn       Milt       Hög  Stark    Nej


### Skriv ut kodningen av kategorierna för respektive kolumn


In [4]:
for catindex, catname in enumerate( df.columns ):
    print( "{0:10s} = {1}".format( catname, df[catname].cat.categories ) )

Dag        = Index(['Dag01', 'Dag02', 'Dag03', 'Dag04', 'Dag05', 'Dag06', 'Dag07', 'Dag08',
       'Dag09', 'Dag10', 'Dag11', 'Dag12', 'Dag13', 'Dag14'],
      dtype='object')
Väder      = Index(['Mulet', 'Regn', 'Sol'], dtype='object')
Temperatur = Index(['Kallt', 'Milt', 'Varmt'], dtype='object')
Fuktighet  = Index(['Hög', 'Normal'], dtype='object')
Vind       = Index(['Stark', 'Svag'], dtype='object')
Grilla     = Index(['Ja', 'Nej'], dtype='object')


### Spara integer-representation av kategorierna (för respektive kolumn) i en ny dataframe, och skriv ut deras koder på skärmen


In [5]:
df_coded = pd.DataFrame()

df_coded['Dag'] = df['Dag'].astype('category').cat.codes
df_coded['Väder'] = df['Väder'].astype('category').cat.codes
df_coded['Temperatur'] = df['Temperatur'].astype('category').cat.codes
df_coded['Fuktighet'] = df['Fuktighet'].astype('category').cat.codes
df_coded['Vind'] = df['Vind'].astype('category').cat.codes
df_coded['Grilla'] = df['Grilla'].astype('category').cat.codes

print( df_coded )

    Dag  Väder  Temperatur  Fuktighet  Vind  Grilla
0     0      2           2          0     1       0
1     1      2           2          0     0       0
2     2      0           2          0     1       0
3     3      1           2          0     1       0
4     4      1           0          1     1       1
5     5      1           0          1     0       1
6     6      0           0          1     1       0
7     7      2           1          0     1       0
8     8      2           0          1     1       0
9     9      1           1          1     1       1
10   10      2           1          1     0       0
11   11      0           1          0     0       1
12   12      0           2          1     1       0
13   13      1           1          0     0       1


### Spara datan i samtliga kolumner _utom_ första och sista i dataframen "X". Spara därefter sista kolumen (alltså klassvariablen) till dataframen "y"


In [6]:
X = df_coded.iloc[:,1:5] 
y = df_coded.iloc[:,5]

### Skapa ett beslutsträd med följande argument satta: criterion="entropy" samt random_state"=0. Bygg därefter beslutsträdet baserat på datan i "X" och "y" med hjälp av beslutsträdets funktion fit()


In [7]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(criterion='entropy', random_state=0)
dtree.fit(X,y)

DecisionTreeClassifier(criterion='entropy', random_state=0)

### Skriv ut trädet till en PNG-fil.

In [8]:
# Generera svartvit trädstruktur till PNG-fil
from graphviz import Source
from sklearn.tree import export_graphviz

graph = Source( export_graphviz(dtree, out_file=None, feature_names=X.columns))
graph.format = 'png'
graph.render('grilla_tree',view=True)

'grilla_tree.png'

In [12]:
# Generarea snyggare färglagd trädstruktur (vita noder, negativa löv som röda och positiva lov som gröna) till PNG-fil.
import pydotplus

dot_data = export_graphviz(dtree, feature_names=X.columns, out_file=None, filled=True, rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)
nodes = graph.get_node_list()
colors =  ('springgreen', 'tomato', 'white')

for node in nodes:
    if node.get_name() not in ('node', 'edge', '"\\n"'):
        values = dtree.tree_.value[int(node.get_name())][0]
        #color only nodes where only one class is present
        if max(values) == sum(values):    
            node.set_fillcolor(colors[np.argmax(values)])
        #mixed nodes get the default color
        else:
            node.set_fillcolor(colors[-1])
graph.write_png('grilla_tree_color.png')

True

### Skapa en ny instans som har följande värden: "Väder"=0, "Temperatur"=0, "Fuktighet"=0, "Vind"=0

In [13]:
test = pd.DataFrame( [[0,0,0,0]], columns = [ "Väder", "Temperatur", "Fuktighet", "Vind" ] )
print( test.to_string(index=False) )

 Väder  Temperatur  Fuktighet  Vind
     0           0          0     0


### Klassificera den nya instansen m.h.a. beslutsträdet och skriv ut vilken klass den får. Förklara varför instansen klassifieras som den gör. Vilken väg genom trädet används?

In [14]:
test_pred = dtree.predict(test)
print( test_pred )

[1]


### Skapa en ny instans som har följande värden: "Väder"=2, "Temperatur"=2, "Fuktighet"=1, "Vind"=1

In [15]:
test = pd.DataFrame( [[2,2,1,1]], columns = [ "Väder", "Temperatur", "Fuktighet", "Vind" ] )
print( test.to_string(index=False) )

 Väder  Temperatur  Fuktighet  Vind
     2           2          1     1


### Klassificera den nya instansen m.h.a. beslutsträdet och skriv ut vilken klass den får. Förklara varför instansen klassifieras som den gör. Vilken väg genom trädet används?

In [16]:
test_pred = dtree.predict(test)
print( test_pred )

[0]
